In [1]:
import os
import sys
from pathlib import Path

# lägg till projektroten (mappen ovanför notebooks/) på sys.path
root_dir = Path().absolute()

if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])

root_dir = str(root_dir) 
print(f"Root dir: {root_dir}")
print("Local environment")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Root dir: /Users/jacobbjareklint/Code/GitHub/hockey-agent
Local environment
Added the following directory to the PYTHONPATH: /Users/jacobbjareklint/Code/GitHub/hockey-agent


In [ ]:
import hopsworks
from config import settings
import requests
import pandas as pd
import util

/opt/homebrew/Caskroom/miniconda/base/envs/hockey-agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY,
    host = settings.HOPSWORKS_HOST
)


2025-12-30 10:55:57,765 INFO: Initializing external client
2025-12-30 10:55:57,766 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-30 10:55:58,606 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3193


In [ ]:
season_ids = util.generate_season_ids(2000)
season_ids[:5], season_ids[-3:]

(['20002001', '20012002', '20022003', '20032004', '20042005'],
 ['20232024', '20242025', '20252026'])

In [ ]:
all_teams = []

for season_id in season_ids:
    try:
        print(f"Hämtar säsong {season_id}")
        df_season = util.fetch_team_for_season(season_id)
        all_teams.append(df_season)
    except Exception as e:
        print(f"Misslyckades för {season_id}: {e}")

teams_df = pd.concat(all_teams, ignore_index=True)
print(teams_df.shape)

Hämtar säsong 20002001
Hämtar säsong 20012002
Hämtar säsong 20022003
Hämtar säsong 20032004
Hämtar säsong 20042005
Hämtar säsong 20052006
Hämtar säsong 20062007
Hämtar säsong 20072008
Hämtar säsong 20082009
Hämtar säsong 20092010
Hämtar säsong 20102011
Hämtar säsong 20112012
Hämtar säsong 20122013
Hämtar säsong 20132014
Hämtar säsong 20142015
Hämtar säsong 20152016
Hämtar säsong 20162017
Hämtar säsong 20172018
Hämtar säsong 20182019
Hämtar säsong 20192020
Hämtar säsong 20202021
Hämtar säsong 20212022
Hämtar säsong 20222023
Hämtar säsong 20232024
Hämtar säsong 20242025
Hämtar säsong 20252026
(764, 25)


In [ ]:
teams_df = teams_df.rename(columns={
    col: util.to_snake(col) for col in teams_df.columns
})
teams_df

,faceoff_win_pct,games_played,goals_against,goals_against_per_game,goals_for,goals_for_per_game,losses,ot_losses,penalty_kill_net_pct,penalty_kill_pct,...,season_id,shots_against_per_game,shots_for_per_game,team_full_name,team_id,team_shutouts,ties,wins,wins_in_regulation,wins_in_shootout
0,0.551730,82.0,211.0,2.57317,233.0,2.84146,27.0,4.0,0.856777,0.841433,...,20002001,27.71951,28.63414,Washington Capitals,15.0,5.0,10.0,41.0,39.0,0.0
1,0.493820,82.0,195.0,2.37804,295.0,3.59756,19.0,3.0,0.875000,0.846875,...,20002001,24.68292,31.58536,New Jersey Devils,1.0,10.0,12.0,48.0,43.0,0.0
2,0.468586,82.0,238.0,2.90243,239.0,2.91463,28.0,7.0,0.839793,0.808786,...,20002001,25.87804,29.68292,Vancouver Canucks,23.0,2.0,11.0,36.0,31.0,0.0
3,0.483333,82.0,245.0,2.98780,188.0,2.29268,41.0,5.0,0.833333,0.817949,...,20002001,29.47560,25.89024,Anaheim Ducks,24.0,6.0,11.0,25.0,21.0,0.0
4,0.519139,82.0,195.0,2.37804,249.0,3.03658,22.0,5.0,0.892030,0.853471,...,20002001,23.57317,30.41463,St. Louis Blues,19.0,10.0,12.0,43.0,37.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,0.495428,38.0,108.0,2.84210,125.0,3.28947,11.0,3.0,0.839622,0.801887,...,20252026,24.50000,32.52631,Carolina Hurricanes,12.0,2.0,NaN,24.0,15.0,4.0
760,0.490594,37.0,103.0,2.78378,107.0,2.89189,11.0,7.0,0.833333,0.824562,...,20252026,25.54054,25.72972,Philadelphia Flyers,4.0,0.0,NaN,19.0,12.0,5.0
761,0.528436,40.0,130.0,3.25000,135.0,3.37500,14.0,6.0,0.827586,0.793104,...,20252026,27.40000,28.47500,Edmonton Oilers,22.0,2.0,NaN,20.0,15.0,0.0
762,0.474583,38.0,120.0,3.15789,122.0,3.21052,15.0,2.0,0.861538,0.830770,...,20252026,26.57894,29.18421,Florida Panthers,13.0,3.0,NaN,21.0,18.0,2.0


In [8]:
fs = project.get_feature_store()

teams_fg = fs.get_or_create_feature_group(
    name="teams",
    description="NHL team information from standings endpoint",
    version=1,
    primary_key=["team_full_name", "season_id"]
)

In [9]:
teams_fg.insert(teams_df)

Uploading Dataframe: 100.00% |██████████| Rows 764/764 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: teams_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/jobs/named/teams_1_offline_fg_materialization/executions


(Job('teams_1_offline_fg_materialization', 'SPARK'), None)